In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import matplotlib.pyplot as plt

In [23]:
web = "https://www.ev-nepal.com/electric-car-price-in-nepal"

# Update this path to your local ChromeDriver location
path = r"/path/to/chromedriver"

In [24]:
service = Service(executable_path = path)
driver = webdriver.Chrome(service = service)
driver.get(web)

This was my first web scrapping experience so tried my best 

In [25]:
#//div[contains@class,"pb-16 md:py-16"]
#//div[contains(@class,"mx-auto max-w-screen-xl gap-5 px-4 md:px-10")]
#//div[contains(@class,"flex relative w-full")]
#//div[contains(@class,"md:flex mt-4 md:mt-8 w-full gap-6")]
#//div[contains(@class,"w-full min-w-0")]
#//div[contains(@class,"w-full")]

#//div[contains(@class,"mt-6 grid gap-x-3 gap-y-4 sm:gap")] #outer
#//div[contains(@class, "group relative border border")] #each car box

##for the name of car company/brand
#//div[contains(@class,"mt-4 flex justify-between")]
#//a[contains(@class, "text-sm text-primary-txt ")] #car company like byd, tata

#for model name and other features there is idk a outer layer 
#//div[contains(@class, "mt-1 flex items-center gap-2 flex-wrap")] #outer layer to get the model and other features
#//p[contains(@class, "text-xl lg:text-lg font-semibold leading-tight")] #model name
#//p[contains(@class, "text-sm lg:text-xs text-primary-txt")] #battery, motor and range

#//div[contains(@class,"mt-3 grid grid-cols-3 gap-4 items-start")] #grid 
#//p[contains(@class, "mt-1 text-base lg:text-sm font-semibold")] #value of batterykWh, motorkW, rangekms

#for price different layer inside outer and each car box 
#//div[contains(@class, "mt-3 flex justify-between")] 
#//p[contains(@class, "text-sm lg:text-xs text-primary-txt")] #price
#//span[contains(@class, "text-xl lg:text-lg font-semibold")] #value


In [26]:
# outer = driver.find_elements('xpath', value = '//div[contains(@class,"mt-6 grid gap-x-3 gap-y-4 sm:gap")] //div[contains(@class, "group relative border border")]')
# outer = '//div[contains(@class,"mt-6 grid gap-x-3 gap-y-4 sm:gap")]//div[contains(@class, "group relative border border")]//div[contains(@class, "mt-1 flex items-center gap-2 flex-wrap")]'


In [27]:
# --- click the Detailed button ---
try:
    wait = WebDriverWait(driver, 10)
    detailed_button = wait.until(
        EC.element_to_be_clickable(
            (By.XPATH, '//*[text()="Detailed" or normalize-space(.)="Detailed"]')
        )
    )
    detailed_button.click()
    print("Clicked the Detailed button")
    time.sleep(2)  # wait for detailed content to load
except:
    print("Detailed button not found or not clickable")
SCROLL_PAUSE_TIME = 2

last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)

    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    
# outer = driver.find_elements('xpath', value = '//div[contains(@class,"mt-6 grid gap-x-3 gap-y-4 sm:gap")] //div[contains(@class, "group relative border border")]')
outer = '//div[contains(@class, "group relative border border")]'

# get all vehicle cards
cars = driver.find_elements(By.XPATH, outer)
print("Total cars found:", len(cars))

car_data = []

#extract model names
for car in cars:
    try:
       brand = car.find_element(
            By.XPATH,
           './/a[contains(@class, "text-sm text-primary-txt hover:text-primary-600 hover:underline transition-colors")]'
        ).text
    except:
       brand = "N/A"
    print(brand)
    try:
        model = car.find_element(
            By.XPATH,
            './/div[contains(@class, "mt-1 flex items-center gap-2 flex-wrap")]//p[contains(@class, "text-xl lg:text-lg font-semibold leading-tight")]'
        ).text
    except:
        model = "N/A"
    print(model)
    # specs of each car battery, motor, range
    try:
        specs = car.find_elements(
            By.XPATH,
            './/div[contains(@class,"mt-3 grid grid-cols-3")]//p[contains(@class,"mt-1 text-base lg:text-sm font-semibold")]'
        )
        battery = specs[0].text if len(specs) > 0 else "N/A"
        motor   = specs[1].text if len(specs) > 1 else "N/A"
        range_  = specs[2].text if len(specs) > 2 else "N/A"

    except:
        battery = motor = range_ = "N/A"

    try:
        price = car.find_element(
            By.XPATH,
            './/div[contains(@class, "mt-3 flex justify-between")]//span[contains(@class, "text-xl lg:text-lg font-semibold")]'
        ).text
    except:
        price = "N/A"

    print(battery, motor, range_,price)

    car_data.append({
        "Brand": brand,
        "Model": model,
        "Battery_kWh": battery,
        "Motor_kW": motor,
        "Range_km": range_,
        "Price": price
    })

Clicked the Detailed button
Total cars found: 147
BYD
Dolphin
44.9 kWh 70 kW 340 kms Rs. 41,15,000
BYD
Atto 3
60.48 kWh 100 kW 420 kms Rs. 67,80,000
Citroen
E-C3
29.2 kWh 42 kW 320 kms Rs. 35,99,000
Dongfeng
Nammi 01
42.3 kWh 70 kW 317 kms Rs. 36,49,000
XPENG
G6
66 kWh 190 kW 435 kms Rs. 84,99,000
Tata
Tiago EV
24 kWh 48 kW 205 kms Rs. 29,99,000
BYD
Atto 1
38.88 kWh 45 kW 300 kms Rs. 32,95,000
Tata
Nexon EV
30.2 kWh 95 kW 312 kms Rs. 40,99,000
Avatr
Avatr
0 kWh 0 kW 0 kms TBA
BAW
Brumby
17.28 kWh 30 kW 205 kms Rs. 19,99,000
BMW
iX1
66.5 kWh 150 kW 474 kms Rs. 1,49,96,000
BMW
iX2
66.5 kWh 150 kW 475 kms Rs. 1,79,96,000
BMW
iX3
80 kWh 210 kW 460 kms Rs. 2,29,96,000
BYD
Atto 1
30.08 kWh 45 kW 230 kms Rs. 28,95,000
BYD
Atto 2
51.13 kWh 100 kW 345 kms Rs. 43,99,000
BYD
Atto 3
49.92 kWh 100 kW 345 kms Rs. 56,90,000
BYD
E6
71.7 kWh 70 kW 522 kms Rs. 59,00,000
BYD
M6
71.8 kWh 100 kW 440 kms Rs. 58,90,000
BYD
M6
55.4 kWh 100 kW 420 kms Rs. 55,00,000
BYD
Seal
61.44 kWh 150 kW 460 kms Rs. 1,10,00

SAVING TO A CSV FILE

In [28]:
df = pd.DataFrame(car_data)
df.to_csv("ev_nepal_cars.csv", index = False)
print("CSV file saved as ev_nepal_cars.csv")
driver.quit()

CSV file saved as ev_nepal_cars.csv
